In [163]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [164]:
model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval() # Sets model to inference mode

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [165]:
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear}, # Only quantize dense(linear) layers
    dtype=torch.qint8 # Quantize weights 
)

In [166]:
torch.save(quantized_model.state_dict(), "QuickTake.pt") # Model saved with tensor parameters

In [167]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# model.load_state_dict(torch.load("QuickTake.pt"))
# model.eval()

In [168]:
article = """
Manitoba is shaking up its moose hunt system after a slash to the number of licences offered last year received both flak and legal challenges from a northern Manitoba First Nation and a provincial conservation organization representing the interests of hunters.

The province will grant 350 moose hunting licences this year, up from the 100 offered last year, Natural Resources and Indigenous Futures Minister Ian Bushie said in a Thursday news release.

The move aligns with the province's obligations under the 1977 Northern Flood Agreement, which states Manitoba must prioritize Indigenous harvesters on traditional territory of Pimicikamak Cree Nation, Bushie said in the release.

Jamie Moses, then the minister of natural resources, sparked fury from Pimicikamak and the Manitoba Wildlife Federation last summer after he cut the number of moose draw hunting licences for Manitoba residents by 75 per cent — from a total of 400 to 100 — across four of Manitoba's 62 game hunting areas.

First Nation's moose hunting case to make 'new case law' in Manitoba, lawyers say as hearings conclude
Manitoba made 'political compromise' by cutting moose draw licences in some areas, lawyer argues
The traditional territory of Pimicikamak, also known as the Cross Lake Resource Area, spans nearly 15,000 square kilometres and portions of four of Manitoba's game hunting areas, including two of four GHAs subject to the 75 per cent licence reduction.

Pimicikamak and the wildlife federation both challenged Moses' decision in court. Manitoba Court of King's Bench Justice Brian Bowman heard from lawyers representing Pimicikamak, the wildlife federation and the province during a two-day hearing last November.

New wildlife advisory board
Pimicikamak's lawyers argued the province's July 11 licensing decision infringed on its rights under provincial laws, Treaty 5 and the 1977 Northern Flood Agreement. The wildlife federation argued the cut in licences was not based on scientific data.

The province's lawyers asked Bowman to dismiss both applications, but to dismiss Pimicikamak's case without prejudice so the First Nation could potentially pursue a lawsuit instead, because while both groups have an interest in Manitoba wildlife, they are not equal, as First Nations people have recognized treaty rights to hunt.

Bowman has yet to deliver a decision in the case.

'The fight's not over,' Pimicikamak chief says after urging non-Indigenous moose hunters to return licences
Pimicikamak Cree Nation asks court to cancel moose hunting licences in territory amid population 'crisis'
The province also says aerial surveys focused on moose were conducted in GHAs 9A and 10 over the winter. Fifteen per cent of GHA 9A, and 12 per cent of GHA 10 has been set aside for exclusive use by Indigenous hunters.

The 350 moose hunting licences being offered this year span the remaining portions of GHAs 9A and 10, as well as GHAs 15 and 15A.

The province is going to work with Pimicikamak to create a wildlife advisory and planning board, which will impact shared wildlife-related resources, Bushie said in the release.

The board will help manage resources and allow discussion of shared land and natural resource management, he said.

"""
# Truncates if longer than 512 tokens
inputs = tokenizer(article, return_tensors="pt", max_length=256, truncation=True)

In [169]:
import time
start_time = time.time()
summary_ids = quantized_model.generate(inputs["input_ids"], max_length=50, min_length=20, do_sample=False)
end_time = time.time()
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary: ", summary)
print("TIme Taken: ", end_time-start_time)

Summary:   The news of a new development development development has been announced announced announced in Canada . The announcement announcement announcement was made to the public, but it is still unclear whether whether or or if if it would be a new report of the new developments
TIme Taken:  1.7509615421295166
